In [23]:
%%sh

pip3 install scipy

algorithm_name=decision-trees-sample


cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

region=$(aws configure get region)
region=${region:-us-west-2}
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

aws ecr describe-repositories --repository-names "${alogorithm_name}" > /dev/null 2<&1

if [ $? -ne 0 ]
then 
aws ecr create-repository --repository-name "${alogorithm_name}" > /dev/null
fi

$(aws ecr get-login --region ${region}  --no-include-email)

if [ -d "/home/ec2-user/Sagemaker" ]; then
sudo service docker restart
fi

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/10 : FROM ubuntu:18.04
 ---> 775349758637
Step 2/10 : FROM python:3
 ---> a2aeea3dfc32
Step 3/10 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> d3fca1661c27
Step 4/10 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> c95401d5bca6
Step 5/10 : RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py &&     pip install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gevent gunicorn &&         (cd /usr/local/lib/python2.7/dist-packages/scipy/.libs; rm *; ln ../../numpy/.libs/* .) &&         rm -rf /root/.cache
 ---> Running in eb1524a705c9
--2019-11-23 01:28:14--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.52.175, 2a04:4e42:d::175
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.52.175|:443... connected.
HTTP

You are using pip version 19.0.2, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

Parameter validation failed:
Invalid length for parameter repositoryName, value: 0, valid range: 2-inf
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

The command '/bin/sh -c wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py &&     pip install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gevent gunicorn &&         (cd /usr/local/lib/python2.7/dist-packages/scipy/.libs; rm *; ln ../../numpy/.libs/* .) &&         rm -rf /root/.cache' returned a non-zero code: 1


In [2]:
import sagemaker as sage
from sagemaker import get_execution_role

In [3]:
sess = sage.Session()

In [4]:
data_location = sess.upload_data('data/creditcard.csv', key_prefix="scikirt-learn")

In [5]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']

In [6]:
region = sess.boto_session.region_name

In [7]:
image = '{}.dkr.ecr.{}.amazonaws.com/decision-trees-sample'.format(account, region)

In [8]:
role = get_execution_role()

In [9]:
tree = sage.estimator.Estimator(image, role, 1, 'ml.c4.2xlarge',
                               output_path="s3://{}/output".format(sess.default_bucket()),
                               sagemaker_session=sess)

In [10]:
tree.fit(data_location)

2019-11-25 20:06:36 Starting - Starting the training job...
2019-11-25 20:06:37 Starting - Launching requested ML instances......
2019-11-25 20:07:37 Starting - Preparing the instances for training...
2019-11-25 20:08:28 Downloading - Downloading input data
2019-11-25 20:08:28 Training - Downloading the training image...
2019-11-25 20:08:50 Training - Training image download completed. Training in progress.Starting the training.
Training complete.

2019-11-25 20:09:19 Uploading - Uploading generated training model
2019-11-25 20:09:19 Completed - Training job completed
Training seconds: 62
Billable seconds: 62


In [37]:
from sagemaker.predictor import csv_serializer 

predictor = tree.deploy (1, 'ml.m4.xlarge',serializer=csv_serializer)

Using already existing model: decision-trees-sample-2019-11-25-20-06-36-532


ClientError: An error occurred (ValidationException) when calling the CreateEndpoint operation: Cannot create already existing endpoint "arn:aws:sagemaker:us-west-2:634051656072:endpoint/decision-trees-sample-2019-11-25-20-06-36-532".

In [33]:
test = sess.upload_data('data/test.csv', key_prefix="scikirt-learn")

In [34]:
import pandas as pd

test = pd.read_csv("data/test.csv")

In [35]:
results = predictor.predict(test.values).decode('utf-8')

In [36]:
results = results.split()
print(results)

['Genuine', 'Genuine', 'Genuine', 'Genuine', 'Genuine', 'Genuine', 'Genuine', 'Fraud', 'Genuine', 'Genuine', 'Fraud', 'Genuine', 'Genuine', 'Genuine', 'Fraud', 'Genuine', 'Genuine', 'Genuine']
